In [36]:
import illustris_python as il

import numpy as np
import pickle
import pandas as pd
import re

In [6]:
base_path = "/Users/users/nastase/PROJECT/"

In [28]:
data_directory = "/Users/users/nastase/PROJECT/DATA/nexus_outputs/MMF_outputs/"

In [31]:
def tempfile_finder(snapnum):
    tempfile = f"/Users/users/nastase/PROJECT/DATA/nexus_outputs/MMF_outputs/output_{snapnum}.a_den"
    
    return tempfile

In [39]:
tempfile_initial = tempfile_finder(135)

# Data Imports

## Illustris

In [32]:
def snapshot_finder(snapnum, base_path=base_path):
    snap = il.snapshot.loadSubset(base_path, 135, 'dm', ["Coordinates", "ParticleIDs"])
    positions_array = snap["Coordinates"]
    ids_array = snap["ParticleIDs"]
    
    return positions_array, ids_array

In [41]:
positions_array_initial, ids_array = snapshot_finder(135)

## NEXUS+ results dataframe

In [8]:
with open(f'/Users/users/nastase/PROJECT/DATA/nexus_outputs/particle_selecta/particle_ids_all.pickle', 'rb') as handle:
    joined_dfs = pickle.load(handle)

## NEXUS+ MMF results

In [29]:
def readNEXUS(densityFile, gridsize=512):
    
    pattern = r'(\d{3})\.a_den$'
    
    # Search for the pattern in the densityFile name
    match = re.search(pattern, densityFile)
    
    if match:
        # Extract the 3 digits
        digits = match.group(1)
#         print("Extracted digits: {digits}")
    else:
        print("No matching digits found before '.a_den'")
        
    shape = (gridsize,gridsize,gridsize)
    MMFn = np.fromfile(f"{data_directory}node_{digits}_clean.MMF",dtype=np.int16)
    MMFn = np.reshape(MMFn[527:-1],shape)
    MMFf = np.fromfile(f"{data_directory}fila_{digits}_clean.MMF",dtype=np.int16)
    MMFf = np.reshape(MMFf[527:-1],shape)
    MMFw = np.fromfile(f"{data_directory}wall_{digits}_clean.MMF",dtype=np.int16)
    MMFw = np.reshape(MMFw[527:-1],shape)
    
    return MMFn, MMFf, MMFw

In [37]:
MMFn, MMFf, MMFw = readNEXUS(tempfile)

# Snapping data to NEXUS+ grid

we want to initially normalise the coordinates, such that they overlap with the grid that comes out of nexus

In [9]:
def snap_to_grid(positions_array, gridsize=512):
    
    positions_grid = ((positions_array - np.min(positions_array)) / (np.max(positions_array) - np.min(positions_array)) * gridsize) - 1
    
    return positions_grid

In [42]:
grid_positions_initial = snap_to_grid(positions_array_initial)

In [43]:
particles_df_initial = pd.DataFrame(grid_positions_initial, columns=['x', 'y', 'z'])
particles_df['ParticleID'] = ids_array

# Extracting particle positions in grid

In [27]:
def position_finder(snapnums, particle_type_i, particle_type_ii, particles_df, selecta_df ):
    """
    finds the positions in the NEXUS sized grid, per environment and per snapshot
    
    selecta_df: df containing all the identified particles
    """
    
    results = {}
    
    initial_snapshot=snapnums[0]
    positions_i, ids = snapshot_finder(initial_snapshot)
    
    particles_df_i = pd.DataFrame(snap_to_grid(positions_i), columns=['x', 'y', 'z'])
    particles_df_i['ParticleID'] = ids
    
    indexing_name_i = f"particle_type_{initial_snapshot}"
    tmp_ids_i = selecta_df[selecta_df[indexing_name_i] == particle_type_i][indexing_name_i].index
    tmp_positions_grid_i = particles_df[particles_df['ParticleID'].isin(tmp_ids_i)][['x', 'y', 'z']]
    
    results[f'{particle_type_i}_positions_{initial_snapshot}'] = tmp_positions_grid_i
    
    for snapshot in snapnums[1:]:
        positions_ii = snapshot_finder(snapshot)[0]
    
    return tmp_positions_grid

In [54]:
snapnumlala = 70

In [56]:
 w_ids_later = joined_dfs[joined_dfs[f"particle_type_{snapnumlala:03}"] == 'w']['particle_type_135'].index

In [61]:
joined_dfs[joined_dfs[f"particle_type_{snapnumlala:03}"] == 'w']['particle_type_135']

15685457    n
87844357    n
15833632    n
15848232    n
15852957    n
           ..
94196365    f
94196366    w
94196369    f
94196373    w
94196375    w
Name: particle_type_135, Length: 31401530, dtype: category
Categories (4, object): ['f', 'n', 'v', 'w']

In [38]:
position_finder(135, 'v', particles_df, joined_dfs)

,x,y,z
9721354,54.393417,83.109436,239.050049
9729448,54.471657,83.619659,239.070404
9731224,54.262611,83.067642,239.199142
9732135,54.410904,83.001785,239.139709
9733466,54.230911,83.046570,239.273895
...,...,...,...
94196359,405.715912,6.611196,506.254883
94196366,407.210022,190.872437,294.781464
94196367,427.440430,217.235443,434.804840
94196370,402.230225,310.110504,188.078995
